In [8]:
from math import sqrt, pi, cos, sin, radians, atan2, asin
import numpy as np
import pickle
import gurobipy as gp
import bisect
import time
import random
from gurobipy import GRB
from networkx import DiGraph, relabel_nodes, set_edge_attributes
from numpy import array
from cspy import Tabu, BiDirectional, GreedyElim

In [9]:
def pos2dist(point1,point2):
    radius = 3958.756;
    lat1 = point1[0]*pi/180
    lat2 = point2[0]*pi/180
    lon1 = point1[1]*pi/180
    lon2 = point2[1]*pi/180
    deltaLat = lat2-lat1;
    deltaLon = lon2-lon1;
    a = sin((deltaLat)/2)**2 + cos(lat1)*cos(lat2) * sin(deltaLon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    x = deltaLon*cos((lat1+lat2)/2)
    y = deltaLat
    d1 = radius*c
    d2 = radius*sqrt(x*x + y*y)
    return d1


def intilization_and_set_up(data_number, ins):
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "tours", "rb") as fp:   # Upickling
        tours = pickle.load(fp)
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "prize", "rb") as fp:   # Unpickling
        prize_cd_vec = pickle.load(fp)
    with open("HSCPP"+str(data_number)+"_ins"+ str(ins) + "IT", "rb") as fp:   # Unpickling
        IT_vec = pickle.load(fp)

    T = len(tours)
    a = [[0] * 80 for _ in range(T)]
    in_tour_vec = {}
    for i in range(80):
        in_tour_vec[i] = []
    for k in range(T):
        for i in tours[k]:
            a[k][i] = 1
            in_tour_vec[i].append(k)
  
    return a, in_tour_vec, tours, prize_cd_vec, IT_vec

In [10]:
def pccrpcgsubproblemheu(Prize_set, inner_set, outer_set, dist_set, depot, dual_alpha, dual_beta, 
                      dual_gamma):
    
    
    G = DiGraph(directed=True, n_res=2)
    mat = [[0] * 81 for _ in range(81)]
    dist_local = [[0] * 81 for _ in range(81)]
    #depot is the source, node 81 is the sink
    #6.5*0.8=5.2 is the speed of the vehicle
    for i in range(80):
        for j in range(81):
            if i != j:
                if i == depot and j < 80:
                    if (j in inner_set):
                        mat[i][j] = (dual_alpha[i] - Prize_set[i])
                        G.add_edge('Source', j, res_cost=array([dist_set[i][j], 0]), weight=mat[i][j])
                    else:
                        mat[i][j] = (dual_alpha[i] - Prize_set[i]) + dual_beta/5.2 * (dist_set[i][j]) 
                        G.add_edge('Source', j, res_cost=array([dist_set[i][j], 1]), weight=mat[i][j])
                    dist_local[i][j] = dist_set[i][j]
                        
                elif i!= depot and j != depot and j < 80:
                    if (i in inner_set and j in inner_set):
                        mat[i][j] = (dual_alpha[i] - Prize_set[i])
                        G.add_edge(i, j, res_cost=array([dist_set[i][j], 0]), weight=mat[i][j])           
                    elif (i in inner_set and j in outer_set):
                        mat[i][j] = (dual_alpha[i] - Prize_set[i]) + dual_beta/5.2 * (dist_set[i][j]) 
                        G.add_edge(i, j, res_cost=array([dist_set[i][j], 1]), weight=mat[i][j])
                    else:
                        mat[i][j] = (dual_alpha[i] - Prize_set[i])  + dual_beta/5.2 * (dist_set[i][j]) 
                        G.add_edge(i, j, res_cost=array([dist_set[i][j], 0]), weight=mat[i][j])
                    dist_local[i][j] = dist_set[i][j]
                elif j == 80 and i != depot:
                    if (i in inner_set):
                        mat[i][j] = (dual_alpha[i] - Prize_set[i])
                        G.add_edge(i, 'Sink', res_cost=array([dist_set[i][depot], 0]), weight=mat[i][j])       
                    else:
                        mat[i][j] = (dual_alpha[i] - Prize_set[i]) + dual_beta/5.2 * (dist_set[i][depot])
                        G.add_edge(i, 'Sink', res_cost=array([dist_set[i][depot], 0]), weight=mat[i][j])
                    dist_local[i][j] = dist_set[i][depot]
                        
    max_res = [20.8, 1]
    min_res = [0, 1]
    bidirec  = BiDirectional(G, max_res, min_res, elementary=True)
    bidirec.run()
    path = bidirec.path  # print route
    path[0] = depot
    path[-1] = depot
    prize_cd = 0
    inner_time = 0
    for i in range(len(path) - 1):
        prize_cd += Prize_set[path[i]]
        if path[i] in inner_set and path[i+1] in inner_set:
            inner_time += dist_set[path[i]][path[i+1]]
    inner_time += (26*0.8-bidirec.consumed_resources[0])
#     print("redcued_cost_csp", -(bidirec.total_cost + dual_gamma - 4*dual_beta), path)
    path.pop()
    return [path], [prize_cd], -(bidirec.total_cost + dual_gamma - 4*dual_beta), [inner_time/5.2]

In [11]:
def applyingcg(a, in_tour_vec, tours, prize_cd_vec, IT_vec, ins):
    master = gp.Model("LP") # master LP problem
    master.ModelSense=-1   # maximize
    z = {}
    T = len(tours)
    for t in range(T):
        z[t] = master.addVar(obj=prize_cd_vec[t], vtype="C", lb = 0)
    master.update()
    nodes={}
    for i in range(80):
        coef = [a[k][i] for k in range(T) if a[k][i] > 0]
        var = [z[k] for k in range(T) if a[k][i] > 0]
        nodes[i] = master.addLConstr(gp.LinExpr(coef,var), "<=", 1)
    IT_const = master.addConstr(sum(-IT_vec[k] * z[k] for k in range(T)) <= -4)
    Vehicles_const = master.addConstr(sum(z[k] for k in range(T))<= 3)
    master.update()   # must update before calling relax()
    master.Params.OutputFlag = 0 # silent mode
    
    
    iter_counter = 0
    start = time.time()
    reduced_cost = []
    inner_len = len(inner)
    Prize_80 = Prize_80_vec[ins]
    while iter_counter < 1000:
        iter_counter += 1
#         print("iter", iter_counter)
        master.optimize()
        aggregate_equal = []
        type_same = []
        for i in range(80):
            for j in range(i+1, 80):
                if in_tour_vec[i] == in_tour_vec[j] and in_tour_vec[i] not in type_same:
                    aggregate_equal.append((i, j))
            type_same.append(in_tour_vec[i])
        dual = gp.Model("dual")
        dual.setParam('OutputFlag', 0)
        alpha_v = dual.addVars(80, vtype=GRB.CONTINUOUS, lb = 0)
        beta_v = dual.addVar(vtype=GRB.CONTINUOUS, lb = 0)
        gamma_v = dual.addVar(vtype=GRB.CONTINUOUS, lb = 0)
        dual.setObjective(alpha_v.sum() - 4 * beta_v + 3 * gamma_v, GRB.MINIMIZE)
        dual.addConstrs(sum(a[k][i] * alpha_v[i] for i in range(80)) - IT_vec[k]*beta_v + gamma_v >= prize_cd_vec[k] for k in range(T))    
        dual.addConstrs(alpha_v[i]/Prize_80[i] == alpha_v[j]/Prize_80[j] for (i, j) in aggregate_equal)
        dual.optimize()
        if abs(master.objVal - dual.objVal) > 1e-6:
            print('error')
            break
        alpha = [0] * 80
        for i in range(82):
            if i < 80:
                alpha[i] = alpha_v[i].X
            elif i == 80:
                beta = beta_v.X
            elif i == 81:
                gamma = gamma_v.X
        if time.time() - start > 3000:
            print('timeup')
            break
        used = []
        key = 0
        while len(used) < inner_len:
            key = random.randint(0, inner_len - 1)
            depot = inner[key]
            if inner[key] not in used:
                used.append(inner[key])
                sol_sub_vec, prize_cd_sub_vec, obj, IT_time_sub_vec =   pccrpcgsubproblemheu(Prize_80, inner, outer_all,
                                                        dist_80, depot, alpha, beta, gamma)
                if obj > 1e-6:
                    break
        if obj < 1e-6:
            print('done')
            break
#         print('obj', obj)
        reduced_cost.append(obj)
        prize_cd_vec += prize_cd_sub_vec
        IT_vec += IT_time_sub_vec
        for i in range(1):
            sol = sol_sub_vec[i]
            tmp = [0] * 80
            for j in range(len(sol)):
                tmp[sol[j]] = 1
                in_tour_vec[sol[j]].append(T)
            a.append(tmp)
            tours.append(sol)
            col = gp.Column()
            for j in sol:
                col.addTerms(1, nodes[j])
            col.addTerms(-IT_time_sub_vec[i], IT_const)
            col.addTerms(1, Vehicles_const)
            z[T] = master.addVar(obj=prize_cd_sub_vec[i], vtype="C", lb = 0, column=col)
            T += 1
            master.update()   # must update before calling relax()
    #     if iter_counter % 10 == 0:
    #         master.write("MP" + str(iter_counter) + ".lp")
    return reduced_cost, prize_cd_vec, IT_vec, a, time.time() - start

In [12]:
def applyingip(prize_cd_vec, IT_vec, a):
    T = len(tours)
    masterip = gp.Model("iP") # master LP problem
    masterip.ModelSense=-1   # maximize
    z_ip = {}
    for t in range(T):
        z_ip[t] = masterip.addVar(obj=prize_cd_vec[t], vtype="B")
    nodes={}
    for i in range(80):
        coef = [a[k][i] for k in range(T) if a[k][i] > 0]
        var = [z_ip[k] for k in range(T) if a[k][i] > 0]
        nodes[i] = masterip.addLConstr(gp.LinExpr(coef,var), "<=", 1)
    masterip.addConstr(sum(-IT_vec[k] * z_ip[k] for k in range(T)) <= -4)
    masterip.addConstr(sum(z_ip[k] for k in range(T))== 3)
    masterip.optimize()
    sol_tour = []
    prize_cd = 0
    prize_cd_sol = []
    IT_sol = []
    L_sol = []
    for t in range(T):
        if z_ip[t].X > 0.5:
            sol_tour.append(t)
            prize_cd += prize_cd_vec[t]
            prize_cd_sol.append(prize_cd_vec[t])
            IT_sol.append(IT_vec[t])
            tmp_tour = tours[t] 
            dist = 0
            for i in range(len(tmp_tour) - 1):
                dist += dist_80[tmp_tour[i]][tmp_tour[i+1]]
            dist += dist_80[tmp_tour[-1]][tmp_tour[0]]
            L_sol.append(dist)
    return sol_tour, prize_cd, L_sol, prize_cd_sol, IT_sol

In [13]:
data_number = 1
K = 3
tau = 26 * 0.8
with open("HSCPP_real_world_" + str(data_number), "rb") as fp:   # Unpickling
    [Prize_80_vec, dist_80, marker_locations] = pickle.load(fp)

nCusts = 80
x0, y0 = 39.16021, -77.20217
outer_all = []
cnt = nCusts
inner = []
for i in range(nCusts):
    if pos2dist((x0,y0), marker_locations[i]) <= 1.4:
        inner.append(i)
    else:
        outer_all.append(i)

In [14]:
prize_cd_final = []
L_sol_vec = []
prize_cd_sol_vec = []
IT_sol_vec = []
for ins in range(1):
    print(ins)
    a, in_tour_vec, tours, prize_cd_vec, IT_vec = intilization_and_set_up(data_number, ins)
    reduced_cost, prize_cd_vec, IT_vec, a, run_time_csp = applyingcg(a, in_tour_vec, tours, prize_cd_vec, IT_vec, ins)
    sol_tour, prize_cd, L_sol, prize_cd_sol, IT_sol = applyingip(prize_cd_vec, IT_vec, a)
    print(run_time_csp)
    prize_cd_final.append(prize_cd)
    L_sol_vec += L_sol
    prize_cd_sol_vec += prize_cd_sol
    IT_sol_vec += IT_sol
# print(prize_cd_final)

0
timeup
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 82 rows, 922 columns and 21109 nonzeros
Model fingerprint: 0xf718a528
Variable types: 0 continuous, 922 integer (922 binary)
Coefficient statistics:
  Matrix range     [4e-02, 4e+00]
  Objective range  [7e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 47.8863549
Presolve removed 9 rows and 209 columns
Presolve time: 0.12s
Presolved: 73 rows, 713 columns, 15846 nonzeros
Variable types: 0 continuous, 713 integer (713 binary)

Root relaxation: objective 4.830325e+01, 348 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      48.1413

In [ ]:
print(prize_cd_final, L_sol_vec,  prize_cd_sol_vec, IT_sol_vec)

In [ ]:
def findingskl(tours, dist_80, IT):
    import bisect
    inner_real_all = []
    outer_all = []
    for i in range(80):
        if pos2dist((x0,y0), marker_locations[i]) <= 1.4:
            inner_real_all.append(i)
        else:
            outer_all.append(i)
    time_to_inner_vec = []
    starting_location = [0] * 3
    for i in range(3):
        tours[i].append(tours[i][0])
        for j in range(len(tours[i])-1):
            if tours[i][j] in inner_real_all and tours[i][j+1] in outer_all:
                break
        if j != 0:
            tours[i] = tours[i][j:] + tours[i][1:j]   
        else:
            tours[i] = tours[i][0:-1]   
    for i in range(3):
        tours[i].append(tours[i][0])
        time_to_inner = [dist_80[tours[i][0]][tours[i][1]]]
        for j in range(1, len(tours[i])-1):
            if tours[i][j] in inner_real_all and tours[i][j+1] in inner_real_all:
                break
            time_to_inner.append(dist_80[tours[i][j]][tours[i][j+1]] + time_to_inner[-1])
        time_to_inner_vec.append(time_to_inner)
        if i == 0:
            print(j)
            starting_location[i] = tours[i][j]
        elif i == 1:
            print(time_to_inner, IT)
            starting_location[i] = tours[i][bisect.bisect_left(time_to_inner, time_to_inner[-1]-IT[0]*5.2)]
        else:
            starting_location[i] = tours[i][0]
    return tours, time_to_inner_vec, starting_location

In [ ]:
tours_selected = [tours[sol_tour[0]]] + [tours[sol_tour[1]]] + [tours[sol_tour[2]]]
IT_time_vec = [IT_vec[sol_tour[0]]] + [IT_vec[sol_tour[1]]] + [IT_vec[sol_tour[2]]]
tours, time_to_inner_vec, starting_location = findingskl(tours_selected, dist_80, IT_time_vec)